# import relevant libraries

In [8]:
import sys
import os
import pinyin
import requests
import pandas as pd
import numpy as np
import re
import math

import site
print('path to packages:')
print(site.getsitepackages())

print('path to pinyin:')
print(os.path.dirname(pinyin.__file__))

# in my local computer, the packages are installed at:
# c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\

path to packages:
['c:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python39', 'c:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages']
path to pinyin:
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pinyin


In [6]:
# install libraries
#!{sys.executable} -m pip install --target='c:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python39\\' pinyin
#!{sys.executable} -m pip install --target='/n/data1/hms/sysbio/hormoz/users/sungchul/lib/' pinyin
#!{sys.executable} -m pip install --target='/n/data1/hms/sysbio/hormoz/users/sungchul/lib/' requests
#!{sys.executable} -m pip install --target='/n/data1/hms/sysbio/hormoz/users/sungchul/lib/' pandas
#!{sys.executable} -m pip install --target='/n/data1/hms/sysbio/hormoz/users/sungchul/lib/' pandas
# sys.path.insert(0,'/n/data1/hms/sysbio/hormoz/users/sungchul/lib/')
# !{sys.executable} -m pip install pinyin

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pinyin


# set up start variables

In [13]:
# for cluster
# path_data = "/n/data1/hms/sysbio/hormoz/users/sungchul/mandarin/"

# for local computer
path_data = 'c:\\Users\\user\\Desktop\\github\\vizman\\data\\'

# df_bufen is the database to all 部分 bufen (radicals) 
# made from this website: https://www.archchinese.com/arch_chinese_radicals.html
df_bufen = pd.read_csv(path_data + "mandarin_bufen.csv")
df_common = pd.read_csv(path_data + "input_common.csv", header = None)
df_bufen_database = pd.read_csv(path_data + "df_bufen_database.csv", index_col = 0)

# old code to initiate data frame
# df_bufen_database = pd.DataFrame([('较','车', '159')])
# df_bufen_database = df_bufen_database.rename(columns = {0:"character", 1:"bufen", 2:"index_bufen"})

In [12]:
os.getcwd()
os.listdir('c:\\Users\\user\\Desktop\\github\\vizman\\data\\')

['df_bufen_database.csv', 'input_common.csv', 'mandarin_bufen.csv']

In [48]:
#df_common
df_bufen_database.to_csv(path_data + "df_bufen_database_test.csv", index = True)
#df_bufen_database_test = pd.read_csv(path_data + "df_bufen_database_test.csv", index_col = 0)
#df_bufen_database = pd.read_csv(path_data + "df_bufen_database.csv", index_col = 0)

In [50]:
df_bufen_database_test

,character,bufen,index_bufen,pinyin,shengdiao
0,较,车,159,jiao,4
1,奥,大,37,ao,4
2,洗,水,85,xi,3
3,教,攴,66,jiao,4
4,戏,戈,62,xi,4
5,弟,弓,57,di,4
6,骗,马,187,pian,4
7,妈,女,38,ma,1
8,十,十,24,shi,2


# functions

In [35]:
def get_bufen(input_char):
    # function: inquires the hanzidb site to extract the radical index and extract the corresponding radical in df_bufen
    # input: input_char is a single mandarin character
    # output: pd.df of input, radical, and radical ID
    URL = "http://hanzidb.org/character/" + input_char
    page = requests.get(URL)
    match = re.search(r'Kangxi radical (\d+)', page.text)
    
    # index_bufen is radical number
    index_bufen = re.search(r'(\d+)', match.group(0)).group(0)
    
    # vec_pinyin = np.vectorize(pinyin.get)(df_bufen_database['character'], format="numerical")
    # vec_shengdiao = np.vectorize(separate_pinyin)(vec_pinyin, output_shengdiao = True)
    # vec_pinyin_unmarked = np.vectorize(separate_pinyin)(vec_pinyin, output_shengdiao = False)
    # df_bufen_database['pinyin'] = vec_pinyin_unmarked
    # df_bufen_database['shengdiao'] = vec_shengdiao

    pinyin_temp = pinyin.get(input_char, format="numerical")
    shengdiao_temp = separate_pinyin(pinyin_temp, output_shengdiao = True)
    pinyin_unmarked = separate_pinyin(pinyin_temp, output_shengdiao = False)

    # return the corresponding bufen in df_bufen
    return(pd.DataFrame(
        [
            [input_char, df_bufen['Radical'][int(index_bufen) - 1], index_bufen, pinyin_unmarked, shengdiao_temp]
        ], 
        columns = [
            "character", "bufen", "index_bufen", 'pinyin', 'shengdiao'
            ]))

def check_bufen_in_data(input_char):
    # function: checks if the input is in the existing df_bufen_database. If it exists, quires df_bufen_database, 
    #   otherwise will inquire the website and add into the database
    global df_bufen_database
    # if character is in the df_bufen_database
    vec_bool = df_bufen_database['character'].str.contains(input_char)
    if(np.sum(vec_bool) == 1):
        print('<check_bufen_in_data> character is in the database')
        index_matching = [i for i, element in enumerate(vec_bool) if element == True]
        index_matching = index_matching[0]
        return(df_bufen_database['bufen'][int(index_matching)])
    else:
        print('<check_bufen_in_data> character is not in the database, adding it')
        #df_bufen_database = df_bufen_database.append(get_bufen(input_char), ignore_index = True)
        result_inquiry = get_bufen(input_char)
        df_bufen_database = pd.concat([df_bufen_database, result_inquiry], ignore_index = True)
        return(result_inquiry['bufen'])       
    
def separate_pinyin(input_str, output_shengdiao = True):
    # function: for a given string of pinyin e.g. jiao3, it separates jiao and 3 and returns either the shengdiao or unmarked pinyin
    if output_shengdiao:
        return(re.findall('[0-9]+', input_str)[0])
    else:
        return(re.findall('[a-zA-Z]+', input_str)[0])

In [40]:
#for input_char in df_common.values[0]:
#    check_bufen_in_data(input_char)
check_bufen_in_data('十')

<check_bufen_in_data> character is not in the database, adding it


0    十
Name: bufen, dtype: object

In [41]:
df_bufen_database

,character,bufen,index_bufen,pinyin,shengdiao
0,较,车,159,jiao,4
1,奥,大,37,ao,4
2,洗,水,85,xi,3
3,教,攴,66,jiao,4
4,戏,戈,62,xi,4
5,弟,弓,57,di,4
6,骗,马,187,pian,4
7,妈,女,38,ma,1
8,十,十,24,shi,2


In [33]:
vec_pinyin = np.vectorize(pinyin.get)(df_bufen_database['character'], format="numerical")
vec_shengdiao = np.vectorize(separate_pinyin)(vec_pinyin, output_shengdiao = True)
vec_pinyin_unmarked = np.vectorize(separate_pinyin)(vec_pinyin, output_shengdiao = False)
df_bufen_database['pinyin'] = vec_pinyin_unmarked
df_bufen_database['shengdiao'] = vec_shengdiao

In [34]:
print(vec_pinyin_unmarked)
print(vec_shengdiao)
print(vec_pinyin)

['jiao' 'ao' 'xi' 'jiao' 'xi' 'di']
['4' '4' '3' '4' '4' '4']
['jiao4' 'ao4' 'xi3' 'jiao4' 'xi4' 'di4']


In [25]:
#df_bufen_database
#check_bufen_in_data('洗')
df_bufen_database
#get_bufen('洗')

,character,bufen,index_bufen,pinyin,shengdiao
0,较,车,159,jiao,4
1,奥,大,37,ao,4
2,洗,水,85,xi,3


In [17]:
n = 21
frame_length = math.ceil(np.sqrt(n))
for i in range(1, n + 1):
    mod_compute_x = min((i) % frame_length, frame_length)
    print(mod_compute_x)
        
min(n % frame_length,5)

1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1


1